In [1]:
from google.colab import files
files.upload()

!rm -r ~/.kaggle
!rm -r ~/content
!mkdir ~/.kaggle
!mkdir ~/content
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d vencerlanz09/sea-animals-image-dataste

import zipfile
zip_ref = zipfile.ZipFile('sea-animals-image-dataste.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

Saving kaggle.json to kaggle.json
rm: cannot remove '/root/.kaggle': No such file or directory
rm: cannot remove '/root/content': No such file or directory
100% 311M/312M [00:18<00:00, 18.8MB/s]
100% 312M/312M [00:18<00:00, 17.5MB/s]


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.applications import ResNet50V2
import tensorflow as tf
import random
import numpy as np
import os
import shutil

In [5]:
dataset_path = "/content/"
train_path = "/content/train"
val_path = "/content/val"
test_path = "/content/test"

if not os.path.exists(train_path):
    os.mkdir(train_path)

if not os.path.exists(val_path):
    os.mkdir(val_path)

if not os.path.exists(test_path):
    os.mkdir(test_path)

In [6]:
IMG_SIZE=256
labels = ['Penguin', 'Starfish', 'Sea Rays', 'Crabs', 'Otter', 'Whale']

In [7]:
for label in labels:
    label_full_path = os.path.join(dataset_path, label)

    images_path = os.listdir(label_full_path)

    images = [i for i in images_path if i.endswith(".jpg")]

    random.shuffle(images)
    split_size = 0.8
    val_size = 0.1
    train_len = int(len(images) * split_size)
    val_len = train_len + int(len(images) * val_size)
    train_images = images[:train_len]
    val_images = images[train_len:val_len]
    test_images = images[val_len:]

    label_train_path = os.path.join(train_path, label)
    label_val_path = os.path.join(val_path, label)
    label_test_path = os.path.join(test_path, label)

    if not os.path.exists(label_train_path):
        os.mkdir(label_train_path)

    if not os.path.exists(label_val_path):
        os.mkdir(label_val_path)

    if not os.path.exists(label_test_path):
        os.mkdir(label_test_path)

    for im in train_images:
        shutil.copy(os.path.join(label_full_path, im), label_train_path)

    for im in val_images:
        shutil.copy(os.path.join(label_full_path, im), label_val_path)

    for im in test_images:
        shutil.copy(os.path.join(label_full_path, im), label_test_path)

In [8]:
batch_size = 32

datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        zoom_range=0.1,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
train_ds = datagen.flow_from_directory(
    train_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    classes=labels,
    batch_size=batch_size
)

val_ds = test_datagen.flow_from_directory(
    val_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    classes=labels,
    batch_size=batch_size
)

test_ds = test_datagen.flow_from_directory(
    test_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    classes=labels,
    batch_size=batch_size
)

Found 2444 images belonging to 6 classes.
Found 302 images belonging to 6 classes.
Found 312 images belonging to 6 classes.


In [19]:
base_model = ResNet50V2(include_top=False, weights='imagenet', input_shape=(IMG_SIZE,IMG_SIZE,3))
base_model.trainable = False
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(len(labels), activation='softmax')
])
model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 8, 8, 2048)        23564800  
                                                                 
 global_average_pooling2d_4  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_12 (Dense)            (None, 256)               524544    
                                                                 
 dropout_6 (Dropout)         (None, 256)               0         
                                                                 
 dense_13 (Dense)            (None, 6)                 1542      
                                                                 
Total params: 24090886 (91.90 MB)
Trainable params: 526086 (2.01 MB)
Non-trainable params: 23564800 (89.89 MB)
_________

In [20]:
model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10
77/77 [==============================] - 49s 585ms/step - loss: 0.3630 - accuracy: 0.8768 - val_loss: 0.2464 - val_accuracy: 0.9338
Epoch 2/10
77/77 [==============================] - 49s 639ms/step - loss: 0.1699 - accuracy: 0.9349 - val_loss: 0.2297 - val_accuracy: 0.9305
Epoch 3/10
77/77 [==============================] - 45s 583ms/step - loss: 0.1413 - accuracy: 0.9472 - val_loss: 0.1801 - val_accuracy: 0.9437
Epoch 4/10
77/77 [==============================] - 46s 592ms/step - loss: 0.1016 - accuracy: 0.9595 - val_loss: 0.1304 - val_accuracy: 0.9536
Epoch 5/10
77/77 [==============================] - 45s 586ms/step - loss: 0.0931 - accuracy: 0.9587 - val_loss: 0.1494 - val_accuracy: 0.9570
Epoch 6/10
77/77 [==============================] - 45s 581ms/step - loss: 0.0887 - accuracy: 0.9648 - val_loss: 0.1638 - val_accuracy: 0.9603
Epoch 7/10
77/77 [==============================] - 44s 566ms/step - loss: 0.0648 - accuracy: 0.9738 - val_loss: 0.1144 - val_accuracy: 0.9636

In [21]:
_, accuracy = model.evaluate(test_ds)
print("Test accuracy:", accuracy)

10/10 [==============================] - 4s 373ms/step - loss: 0.1349 - accuracy: 0.9359
Test accuracy: 0.9358974099159241
